In [1]:
import pandas as pd
from tqdm.contrib.concurrent import process_map
import pathlib
import duckdb


/home/charubaiel/.cache/pypoetry/virtualenvs/mlopsini-xAkA-eId-py3.8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with duckdb.connect('../data/protodb.db') as conn:
    df = conn.query('select * from intel.cian').df()

In [6]:
df.shape

(9907, 32)

In [5]:
df.drop_duplicates(subset=['url','price']).to_parquet('../data/sample_data.parquet')

In [7]:
sample_ = pd.read_parquet('../data/sample_data.parquet')

In [9]:
sample_.shape

(9305, 32)

In [54]:
# with duckdb.connect('../data/protodb.db') as conn:
#     conn.execute('create schema if not exists intel ')
#     conn.execute('create table if not exists intel.cian as select * from sample_')
#     df = conn.query('select * from intel.cian').df()

In [4]:
from dagster import Config
class ConfigMLFlow(Config):

    MLFLOW_URI: str = 'localhost'
    MLFLOW_PORT: int = None



In [7]:
import mlflow

In [2]:
import boto3
import os

In [8]:
my_config = {
        'endpoint_url':'http://localhost:9000',
        'service_name':'s3',
        'AWS_ACCESS_KEY_ID'.lower():'mlflow_user',
        'AWS_SECRET_ACCESS_KEY'.lower():'mlflow_password'
        
    }


In [24]:

s3 = boto3.client(**my_config)

mlflow_s3_bucket = os.getenv('AWS_BUCKET_NAME')



In [30]:
s3.list_objects(Bucket='mlflow')

{'ResponseMetadata': {'RequestId': '176005DF5D3AE634',
  'HostId': 'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-length': '2172',
   'content-security-policy': 'block-all-mixed-content',
   'content-type': 'application/xml',
   'server': 'MinIO',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'Origin, Accept-Encoding',
   'x-amz-id-2': 'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855',
   'x-amz-request-id': '176005DF5D3AE634',
   'x-content-type-options': 'nosniff',
   'x-xss-protection': '1; mode=block',
   'date': 'Wed, 17 May 2023 19:45:17 GMT'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Marker': '',
 'Contents': [{'Key': '2/fdcd5ee59c294b7fb76eee59d659d95f/artifacts/models/MLmodel',
   'LastModified': datetime.datetime(2023, 5, 17, 19, 25, 13, 649000, tzinfo=tzutc()),
   'ETag': '"28901e455276e148474491b443c7a693"',
   'Si

In [27]:
for key in s3.list_objects(Bucket='mlflow')['Contents']:
    print(key['Key'])

2/fdcd5ee59c294b7fb76eee59d659d95f/artifacts/models/MLmodel
2/fdcd5ee59c294b7fb76eee59d659d95f/artifacts/models/conda.yaml
2/fdcd5ee59c294b7fb76eee59d659d95f/artifacts/models/model.cb
2/fdcd5ee59c294b7fb76eee59d659d95f/artifacts/models/python_env.yaml
2/fdcd5ee59c294b7fb76eee59d659d95f/artifacts/models/requirements.txt


In [5]:
mlflow_s3_bucket

'mlflow'

In [ ]:

def init_mlflow():
   